## Decision Tree V2
- all datetime feature
- random cross validation

**Hyperparameter for tweeking:**
- maximale Baumtiefe
- Mindestanzahl an Daten pro Blatt
- Mindestanzahl an Daten pro Knoten
- Features

**Regularisierung:**
- Pruning

**Metriken:**
- R2 score
- mean_absolute_error
- mean_squared_error
- median_absolute_error

#### **_PREPARATION_**

#### 1. FINE DUST (pm10 only) DATA PREP

In [1]:
# GET ALL THE JSONS INTO ONE DATAFRAME
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json
import glob

In [2]:
# Set the search path for files (assuming the directory is relative to the current script)
file_path_mc124 = os.path.join("..", "mc124_data", "*.json")
files = glob.glob(file_path_mc124)

# Create empty list to store dataframes
li_all_files = []

# Loop through list of files and read each one into a dataframe and append to list
for f in files:
    # Read in json
    temp_df = pd.read_json(f)
    # Append df to list
    li_all_files.append(temp_df)

# Optionally concatenate all dataframes into one if needed
if li_all_files:
    combined_df = pd.concat(li_all_files)
    print(f'Combined dataframe shape: {combined_df.shape}')
else:
    print('No dataframes were created.')

Combined dataframe shape: (542555, 6)


In [3]:
combined_df.sample()

,datetime,station,core,component,period,value
1353,2021-10-20T17:00:00+02:00,mc124,no,no_1h,1h,26.0


In [4]:
# FILTER FOR PM10 ONLY
combined_df_pm10 = combined_df[(combined_df['core'] == 'pm10')]
combined_df_pm10.sample(4)

,datetime,station,core,component,period,value
1165,2018-04-21 06:00:00+02:00,mc124,pm10,pm10_1h,1h,40.0
460,2024-05-27 14:00:00+02:00,mc124,pm10,pm10_1h,1h,16.0
495,2020-10-27T20:00:00+01:00,mc124,pm10,pm10_1h,1h,15.0
820,2016-09-22 10:00:00+02:00,mc124,pm10,pm10_1h,1h,26.0


In [5]:
# FILTER BY PARTICLE AND ONLY KEEP THE DATETIME, STATION, PERIOD AND VALUE FEATURE SINCE THE REST ARE CONSTANT INFORMATION (station, core, component, period)
df_reduced = combined_df_pm10[['datetime', 'station', 'core', 'value']]
df_reduced.sample(3)

# CUT OFF THE TIMEZONE INFORMATION FROM THE DATETIME TO AVOID CONVERSION ISSUES DUE TO TIME CHANGE IN MARCH AND OCTOBER
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].astype(str).str.slice(0, 19)
#df_pm10_reduced.loc[:, 'datetime'] = pd.to_datetime(df_pm10_reduced['datetime'], format='mixed')
df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')
df_reduced.loc[:, 'datetime'] = df_reduced['datetime'].dt.tz_localize(None)
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72314 entries, 0 to 3650
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  72314 non-null  datetime64[ns]
 1   station   72314 non-null  object        
 2   core      72314 non-null  object        
 3   value     71471 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 2.8+ MB


C:\Users\a_n_n\AppData\Local\Temp\ipykernel_16112\4237958541.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['datetime'] = pd.to_datetime(df_reduced['datetime'], format='mixed')


In [6]:
df_reduced.sample(3)

,datetime,station,core,value
580,2021-10-27 03:00:00,mc124,pm10,17.0
1945,2020-05-15 18:00:00,mc124,pm10,15.0
916,2016-04-21 10:00:00,mc124,pm10,27.0


In [7]:
#use loc to add it to every for every row
df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)
df_reduced['day'] = df_reduced['datetime'].dt.strftime('%d')  # Day of the month (01-31)
df_reduced['month'] = df_reduced['datetime'].dt.strftime('%m')  # Month (01-12)
df_reduced['year'] = df_reduced['datetime'].dt.strftime('%Y')  # Month (01-12)
df_reduced.sample(3)

C:\Users\a_n_n\AppData\Local\Temp\ipykernel_16112\493841492.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reduced['hour'] = df_reduced['datetime'].dt.strftime('%H')  # Hour (00-23)


,datetime,station,core,value,hour,day,month,year
2105,2023-02-11 10:00:00,mc124,pm10,41.0,10,11,02,2023
145,2018-10-30 18:00:00,mc124,pm10,17.0,18,30,10,2018
2025,2021-09-14 02:00:00,mc124,pm10,15.0,02,14,09,2021


In [8]:
# add day of the week to dataframe
import calendar

days = {
    0: "Monday",
    1: "Tuesday",
    2: "Wednesday",
    3: "Thursday",
    4: "Friday",
    5: "Saturday",
    6: "Sunday",
}

df_daytime = df_reduced
# convert the 'day', 'month', and 'year' columns to integers
df_daytime['day'] = df_reduced['day'].astype(int)
df_daytime['month'] = df_reduced['month'].astype(int)
df_daytime['year'] = df_reduced['year'].astype(int)

# function to determine the day of the week
def get_day_of_week(row):
    return calendar.weekday(row['year'], row['month'], row['day'])

# function to determine if day is weekday or weekend 
def is_weekend(day_number):
    return 1 if day_number >= 5 else 0
    #day_number >= 5 ? 1 : 0
    
# apply the functions to create the new columns
df_daytime['day_of_week'] = df_daytime.apply(get_day_of_week, axis=1)
df_daytime['is_weekend'] = df_daytime['day_of_week'].apply(is_weekend)

df_daytime.rename(columns={'value': 'pm10_value'}, inplace=True)

df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72314 entries, 0 to 3650
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     72314 non-null  datetime64[ns]
 1   station      72314 non-null  object        
 2   core         72314 non-null  object        
 3   pm10_value   71471 non-null  float64       
 4   hour         72314 non-null  object        
 5   day          72314 non-null  int32         
 6   month        72314 non-null  int32         
 7   year         72314 non-null  int32         
 8   day_of_week  72314 non-null  int64         
 9   is_weekend   72314 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 5.2+ MB


In [9]:
df_daytime.dropna(inplace=True)
df_daytime.info()

<class 'pandas.core.frame.DataFrame'>
Index: 71471 entries, 0 to 3650
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   datetime     71471 non-null  datetime64[ns]
 1   station      71471 non-null  object        
 2   core         71471 non-null  object        
 3   pm10_value   71471 non-null  float64       
 4   hour         71471 non-null  object        
 5   day          71471 non-null  int32         
 6   month        71471 non-null  int32         
 7   year         71471 non-null  int32         
 8   day_of_week  71471 non-null  int64         
 9   is_weekend   71471 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int32(3), int64(2), object(3)
memory usage: 5.2+ MB


In [10]:
df_daytime.head()

,datetime,station,core,pm10_value,hour,day,month,year,day_of_week,is_weekend
0,2016-03-31 23:00:00,mc124,pm10,16.0,23,31,3,2016,3,0
4,2016-03-31 22:00:00,mc124,pm10,22.0,22,31,3,2016,3,0
8,2016-03-31 21:00:00,mc124,pm10,22.0,21,31,3,2016,3,0
12,2016-03-31 20:00:00,mc124,pm10,24.0,20,31,3,2016,3,0
16,2016-03-31 19:00:00,mc124,pm10,21.0,19,31,3,2016,3,0


In [11]:
df_daytime.tail(5)

,datetime,station,core,pm10_value,hour,day,month,year,day_of_week,is_weekend
3630,2024-05-01 04:00:00,mc124,pm10,30.0,04,1,5,2024,2,0
3635,2024-05-01 03:00:00,mc124,pm10,29.0,03,1,5,2024,2,0
3640,2024-05-01 02:00:00,mc124,pm10,28.0,02,1,5,2024,2,0
3645,2024-05-01 01:00:00,mc124,pm10,28.0,01,1,5,2024,2,0
3650,2024-05-01 00:00:00,mc124,pm10,29.0,00,1,5,2024,2,0


#### 2. ADDING WIND DATA TO THE DATAFRAME

In [12]:
file_path_berlin = os.path.join("..", "..", "winddaten_berlin","produkt_wind_399_akt.txt")# join because different os use either \ or / as file path seperators
weather_station = pd.read_csv(file_path_berlin, names=['stations_id','date','quality_level','structure_version', 'wind_speed', 'wind_direction', 'eor'], skiprows=1, sep=';') 
# TODO rename names
weather_station.sample(5)

,stations_id,date,quality_level,structure_version,wind_speed,wind_direction,eor
39403,399,2020090222,2,0,9.0,300,eor
21837,399,2018021811,2,0,1.8,120,eor
12996,399,2017021305,2,0,7.7,110,eor
15753,399,2017060822,2,0,8.3,190,eor
44008,399,2021031600,2,0,12.1,310,eor


In [13]:
# Convert the 'dates' column to datetime and store it in a new column 'datetime'
weather_station.loc[:,'datetime'] = pd.to_datetime(weather_station['date'], format='%Y%m%d%H')
weather_station.sample(5)

,stations_id,date,quality_level,structure_version,wind_speed,wind_direction,eor,datetime
31891,399,2019041501,2,0,14.2,80,eor,2019-04-15 01:00:00
6106,399,2016050113,2,0,6.8,60,eor,2016-05-01 13:00:00
50125,399,2021112922,2,0,9.5,300,eor,2021-11-29 22:00:00
9041,399,2016090110,2,0,6.4,300,eor,2016-09-01 10:00:00
32220,399,2019042818,2,0,7.5,300,eor,2019-04-28 18:00:00


In [14]:
# Filter neccessary rows
weather_station_reduced = weather_station[['datetime','wind_speed','wind_direction']]
weather_station_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65861 entries, 0 to 65860
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        65861 non-null  datetime64[ns]
 1   wind_speed      65861 non-null  float64       
 2   wind_direction  65861 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.5 MB


In [15]:
weather_station_reduced.sample(5)

,datetime,wind_speed,wind_direction
15072,2017-05-10 17:00:00,5.2,290
23366,2018-04-23 04:00:00,12.9,260
3343,2016-01-07 09:00:00,9.3,110
65262,2024-05-19 02:00:00,3.1,150
13682,2017-03-13 19:00:00,4.1,200


In [16]:
df_merged = pd.merge(df_reduced, weather_station_reduced, on='datetime', how='outer')
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77137 entries, 0 to 77136
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        77137 non-null  datetime64[ns]
 1   station         71471 non-null  object        
 2   core            71471 non-null  object        
 3   pm10_value      71471 non-null  float64       
 4   hour            71471 non-null  object        
 5   day             71471 non-null  float64       
 6   month           71471 non-null  float64       
 7   year            71471 non-null  float64       
 8   day_of_week     71471 non-null  float64       
 9   is_weekend      71471 non-null  float64       
 10  wind_speed      65861 non-null  float64       
 11  wind_direction  65861 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 7.1+ MB


In [17]:
df_merged.sample(5)

,datetime,station,core,pm10_value,hour,day,month,year,day_of_week,is_weekend,wind_speed,wind_direction
43631,2021-03-09 18:00:00,mc124,pm10,27.0,18,9.0,3.0,2021.0,1.0,0.0,2.8,170.0
31503,2019-11-29 03:00:00,mc124,pm10,9.0,03,29.0,11.0,2019.0,4.0,0.0,12.5,280.0
60045,2023-02-24 10:00:00,mc124,pm10,33.0,10,24.0,2.0,2023.0,4.0,0.0,NaN,NaN
28723,2019-07-23 04:00:00,mc124,pm10,25.0,04,23.0,7.0,2019.0,1.0,0.0,NaN,NaN
38332,2020-08-18 20:00:00,mc124,pm10,20.0,20,18.0,8.0,2020.0,1.0,0.0,4.2,320.0


In [18]:
#Deleting all rows before March 2016 since useful pm10 data starts in March 2016
#TODO check when other particles where first measured
start_date = '2016-03-01'
start_date = pd.Timestamp(start_date)
df_filtered = df_merged[df_merged['datetime'] >= start_date]
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72507 entries, 0 to 77136
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        72507 non-null  datetime64[ns]
 1   station         71471 non-null  object        
 2   core            71471 non-null  object        
 3   pm10_value      71471 non-null  float64       
 4   hour            71471 non-null  object        
 5   day             71471 non-null  float64       
 6   month           71471 non-null  float64       
 7   year            71471 non-null  float64       
 8   day_of_week     71471 non-null  float64       
 9   is_weekend      71471 non-null  float64       
 10  wind_speed      61231 non-null  float64       
 11  wind_direction  61231 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 7.2+ MB


In [19]:
df_filtered.dropna(inplace=True)
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 60195 entries, 0 to 71470
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   datetime        60195 non-null  datetime64[ns]
 1   station         60195 non-null  object        
 2   core            60195 non-null  object        
 3   pm10_value      60195 non-null  float64       
 4   hour            60195 non-null  object        
 5   day             60195 non-null  float64       
 6   month           60195 non-null  float64       
 7   year            60195 non-null  float64       
 8   day_of_week     60195 non-null  float64       
 9   is_weekend      60195 non-null  float64       
 10  wind_speed      60195 non-null  float64       
 11  wind_direction  60195 non-null  float64       
dtypes: datetime64[ns](1), float64(8), object(3)
memory usage: 6.0+ MB


C:\Users\a_n_n\AppData\Local\Temp\ipykernel_16112\1129836580.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered.dropna(inplace=True)


In [20]:
df_filtered.head()

,datetime,station,core,pm10_value,hour,day,month,year,day_of_week,is_weekend,wind_speed,wind_direction
0,2016-03-31 23:00:00,mc124,pm10,16.0,23,31.0,3.0,2016.0,3.0,0.0,8.3,40.0
1,2016-03-31 22:00:00,mc124,pm10,22.0,22,31.0,3.0,2016.0,3.0,0.0,8.4,40.0
2,2016-03-31 21:00:00,mc124,pm10,22.0,21,31.0,3.0,2016.0,3.0,0.0,7.5,40.0
3,2016-03-31 20:00:00,mc124,pm10,24.0,20,31.0,3.0,2016.0,3.0,0.0,9.1,60.0
4,2016-03-31 19:00:00,mc124,pm10,21.0,19,31.0,3.0,2016.0,3.0,0.0,8.6,40.0


In [21]:
df_filtered.tail()

,datetime,station,core,pm10_value,hour,day,month,year,day_of_week,is_weekend,wind_speed,wind_direction
71466,2024-05-01 04:00:00,mc124,pm10,30.0,04,1.0,5.0,2024.0,2.0,0.0,16.1,130.0
71467,2024-05-01 03:00:00,mc124,pm10,29.0,03,1.0,5.0,2024.0,2.0,0.0,13.2,140.0
71468,2024-05-01 02:00:00,mc124,pm10,28.0,02,1.0,5.0,2024.0,2.0,0.0,13.2,140.0
71469,2024-05-01 01:00:00,mc124,pm10,28.0,01,1.0,5.0,2024.0,2.0,0.0,12.7,150.0
71470,2024-05-01 00:00:00,mc124,pm10,29.0,00,1.0,5.0,2024.0,2.0,0.0,14.1,160.0


#### **_ACTUAL MODEL TRAINING_**


#### Part 1: create decision tree on everything

In [22]:
# create features and to be predicted value
y = df_filtered['pm10_value']
X = df_filtered[['hour', 'year', 'day_of_week', 'is_weekend', 'wind_speed', 'wind_direction']]

# SCALING?

In [23]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import tree

In [38]:
dt = DecisionTreeRegressor(max_depth=10)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.31437837059489837

In [39]:
dt.score(X_test, y_test)

0.08161732101911523

In [36]:
dt = DecisionTreeRegressor(max_depth=32)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=1)
dt.fit(X_train, y_train)
dt.score(X_train, y_train)

0.9972633412884366

In [37]:
dt.score(X_test, y_test)

-0.4301104260329449

In [34]:
dt.tree_.max_depth

32

In [35]:
# cross validate
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_absolute_error, mean_squared_error, median_absolute_error, root_mean_squared_error

# Create a DecisionTreeRegressor
decision_tree = DecisionTreeRegressor(max_depth=30)

# Define your scoring functions
scorers = {
    'mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
    'median_absolute_error': make_scorer(median_absolute_error, greater_is_better=False),
    'root_mean_squared_error': make_scorer(root_mean_squared_error, greater_is_better=False),
    'mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False)
}

# Perform cross-validation
# scores = cross_validate(decision_tree, X, y, cv=5, scoring=scorers, return_train_score=True)
scores = cross_validate(decision_tree, X, y, cv=5, scoring=scorers)

# Print the results
print("Cross-validation scores:")
for scorer, score in scores.items():
    print(f"{scorer}: {np.mean(score):.2f} (+/- {np.std(score):.2f})")

Cross-validation scores:
fit_time: 0.23 (+/- 0.02)
score_time: 0.01 (+/- 0.01)
test_mean_absolute_error: -12.32 (+/- 0.81)
test_median_absolute_error: -8.80 (+/- 0.75)
test_root_mean_squared_error: -18.90 (+/- 1.13)
test_mean_squared_error: -358.65 (+/- 42.87)


In [42]:
# average pm10 value for comparison to the error rates
average_pm10 = df_daytime['pm10_value'].mean()
print(f"Average PM10 value: {average_pm10:.2f}")

Average PM10 value: 22.70


In [43]:
median_pm10= df_daytime['pm10_value'].median()
print(f"Median PM10 value: {median_pm10:.2f}")

Median PM10 value: 20.00


In [31]:
print('rerun')

rerun
